# Taller 2 Parte 1

Page to be scraped: El Espectador

In [21]:
# Importing required libraries
import time
import pymongo
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from bs4 import BeautifulSoup
from selenium import webdriver

from pymongo import MongoClient

In [22]:
# Creating a connection to MongoDB
client = MongoClient('localhost', 27017)
db = client['TallerDos']
collection = db['colombia']

In [23]:
# Base URL of the site to be analyzed
SITE_URL = 'https://www.elespectador.com'

In [24]:
# Firefox web driver path
# Download the driver for you S.O. here: https://github.com/mozilla/geckodriver/releases
DRIVER_PATH = './geckodriver.exe'

In [25]:
# Creating a new firefox window
browser = webdriver.Firefox(executable_path = DRIVER_PATH)

C:\Users\candi\AppData\Local\Temp/ipykernel_15260/1910320475.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Firefox(executable_path = DRIVER_PATH)


In [26]:
def make_request(browser, relative_path):
    # Making the request and rendering the browser
    browser.get(SITE_URL + relative_path)
    
    # Simulating vertical scrolling for handling lazy load
    check_height = browser.execute_script('return document.body.scrollHeight;')
    while True:
        browser.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(3)
        height = browser.execute_script('return document.body.scrollHeight;')
        if height == check_height: 
            break 
        check_height = height
    
    # Getting HTML content and passing it to BeautifulSoup for scraping analysis
    return BeautifulSoup(browser.page_source, 'html.parser')

In [27]:
# Getting HTML content for a particular news listing page
soup = make_request(browser, '/archivo/colombia/26/')

In [28]:
# Finding the section where news are contained 
layout = soup.find(class_ = 'Layout-flexAds')

In [29]:
# Getting blocks from layout
blocks = layout.find_all(class_ = 'Container Block', recursive = True)
print(len(blocks))

2


In [30]:
# Finding and concatenating news cards
cards = blocks[0].find_all(class_ = 'Card_rowCardLeft') + blocks[1].find_all(class_ = 'Card_rowCardLeft')
len(cards)

12

In [31]:
# Building a list with title and relative path of the news founded

news = []

for card in cards:
    news.append({
        'title': card.find('h2', class_ = 'Card-Title').find('a').get_text(),
        'relative_path': card.find('h2', class_ = 'Card-Title').find('a')['href']
    })

In [32]:
news

[{'title': 'Tres uniformados heridos en ataque civil a base militar en Norte de Santander',
  'relative_path': '/judicial/tres-uniformados-heridos-en-ataque-civil-a-base-militar-en-norte-de-santander/'},
 {'title': 'Tembló en Colombia: sismo de 4,2 grados',
  'relative_path': '/colombia/temblo-en-colombia-sismo-de-42-grados/'},
 {'title': 'La salud mental, ¿un tema para sacar en medio de un debate político?',
  'relative_path': '/opinion/la-salud-mental-un-tema-para-sacar-en-medio-de-un-debate-politico/'},
 {'title': 'Más que un corte: la historia Barber Art, colectivo de barbería',
  'relative_path': '/colombia/mas-regiones/mas-que-un-corte-la-historia-barber-art-colectivo-de-barberia/'},
 {'title': '“A nadie le importa la vida de los LGBTI”: Wilson Castañeda',
  'relative_path': '/colombia/barranquilla/a-nadie-le-importa-la-vida-de-los-lgbti-wilson-castaneda/'},
 {'title': 'Defensoría pide redoblar esfuerzos para evitar emergencias por ola invernal',
  'relative_path': '/colombia/mas

In [130]:
for n in news:
    # Getting HTML content for each news page
    soup = make_request(browser, n['relative_path'])
    
    # Extracting news metadata
    n['datetime'] = soup.find(class_ = 'ArticleHeader-Date').get_text()
    n['author'] = soup.find(class_ = 'ACredit-Author').find('a').get_text()
    n['summary'] = soup.find(class_ = 'ArticleHeader-Hook').find('div').get_text()
    
    # Extracting and concatenating news full text
    paragraphs = soup.find_all(class_ = 'font--secondary')
    n['full_text'] = ' '.join([p.get_text() for p in paragraphs])

AttributeError: 'NoneType' object has no attribute 'get_text'

In [131]:
news

[{'title': 'Jardín Botánico de Bogotá anunció su segunda ruta agroecológica ',
  'relative_path': '/bogota/jardin-botanico-de-bogota-anuncio-su-segunda-ruta-agroecologica/',
  'datetime': '28 mar 2022  - 2:13 p.\xa0m.',
  'author': 'Redacción Bogotá',
  'summary': 'Esta iniciativa se compone de un circuito de cinco huertas urbanas con historia, patrimonio, mitos y muchas actividades que los ciudadanos podrán compartir. Conozca cómo inscribirse.',
  'full_text': 'El Jardín Botánico de Bogotá junto con el Instituto Distrital de Turismo (IDT) anunció que llevará a cabo su segunda ruta agroecológica “De huerta en huerta”, una iniciativa que apunta al posicionamiento de la actividad agroecológica urbana y periurbana desde una perspectiva turística, promoviendo el conocimiento de las huertas como espacios de interacción. Según la entidad, este proyecto es una apuesta del Plan de Desarrollo Distrital que busca promover nuevas oportunidades de crecimiento económico para los agricultores urbano

In [132]:
# Storing extracted information for further analysis
collection.insert_many(news)